# CODICE PROGETTO 32 OPTIMIZATION 


### Implementazione algoritmo risolutivo

In [1]:
import numpy as np
import random
import time
from scipy.optimize import *

# !pip install pyMCFSimplex
from pyMCFSimplex import *

#### 1. Estrazione dei dati dai file .dmx e .qfc utilizzati per la generazione delle istanze del problema
La prima funzione *matrice_Q(nome_file)* serve per estrarre i costi quadratici che si trovano nella diagonale di Q,

mentre la seconda funzione *leggi_file_dimacs(nome_file)* serve per estrarre le seguenti quantità: 
 - u, b, q
 - numero nodi
 - numero archi 

In [2]:
def matrice_Q(nome_file_qfc):
    vettore = []
    with open(nome_file_qfc, 'r') as file:
        dimensione = int(file.readline()) # la prima riga è la dimensione del vettore
        valori = file.readline().split() # dalla seconda riga 
        vettore = [float(valore) for valore in valori]
        
        #error check 
        if len(vettore) != dimensione:
            raise ValueError("Il numero di valori nel file non corrisponde alla dimensione specificata")
    
    dimensione = len(vettore)
    matrice = np.zeros((dimensione, dimensione)) # creazione matrice 
    np.fill_diagonal(matrice, vettore)  # si riempie diagonale
    
    return matrice, vettore


def leggi_file_dimacs(nome_file):
    numero_nodi = 0
    numero_archi = 0
    u = []
    b = []
    q = []
    from_=[]
    to_=[]
    edges = []

    with open(nome_file, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) > 0:
                if parts[0] == 'p':
                    # legge il numero di nodi e archi dal problema
                    numero_nodi = int(parts[2])
                    numero_archi = int(parts[3])
                    # inizializza il vettore di supply con zeri
                    b = [0] * numero_nodi
                elif parts[0] == 'n':
                    # legge i valori di supply per i nodi
                    nodo_id = int(parts[1])
                    supply = int(parts[2])
                    # assegna il valore di supply al nodo corrispondente
                    b[nodo_id - 1] = supply
                elif parts[0] == 'a':
                    # leggi l'arco e il suo cotso
                    from_node = int(parts[1])
                    to_node = int(parts[2])
                    max_capacity = int(parts[4])
                    costo = int(parts[5])  # leggiamo costo corretto
                    from_.append(from_node)
                    to_.append(to_node)
                    u.append(max_capacity)
                    q.append(costo)
                    edges.append((from_node , to_node ))

    return numero_nodi, numero_archi, u, b, q, edges,from_, to_

#### 2. Algoritmo

Step: 

0. Inizializzazione di x<sub>0</sub> con generazione casuale soggetta al vincolo 0 &le; x<sub>0</sub> &le; u
1. Calcolo del gradiente, risoluzione del sottoproblema lineare usando il solver pyMCFSimplex (x̄), determinazione della direzione
2. Determinazione dello step size &alpha;
3. Aggiornamento della posizione e check terminazione sul prodotto scalare (<grad, d>)

In [4]:
# Questa funzione crea un nuovo file .dmx in cui i valori delle righe che iniziano con 'a' (definizione archi)
# vengono sostituiti con i valori presenti nel vettore gradient. L'obiettivo è modificare il file di input 
# (input_file) in modo da utilizzare al posto dei costi lineari del file originale, il gradiente 
# (vogliamo risolvere il sottoproblema lineare, aka l'approssimazione lineare del problema Taylor 1st ordine).   

def modify_file_with_gradient(input_file, output_file, gradient):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # inizalizzazione dell'indice per il vettore gradient
    gradient_index = 0

    # modifica delle righe che iniziano con 'a'
    for i in range(len(lines)):
        if lines[i].startswith('a'):
            words = lines[i].split()
            words[-1] = str(gradient[gradient_index])
            gradient_index += 1
            lines[i] = ' '.join(words)

    # creazione nuovo file
    with open(output_file, 'w') as file:
        file.writelines(lines)

In [41]:
def algoritmo(nome_file, epsilon, max_iter, Q, q, u, numero_archi):
    k = 0 
    alpha = 2/(2+k)

    start_time = time.time()
    # prodotto_scalare = float("inf")

    # STEP 0 : inizializzazione di x_0
    seed = 42
    np.random.seed(seed)
    x_old = []

    for u_i in u:
        x_i = random.randint(0, u_i)
        x_old.append(x_i)


    while k < max_iter or prodotto_scalare >= epsilon:
        
        # STEP 1 : calcolo gradiente
        gradient = (2 * np.dot(Q, x_old)) + q
        gradient = gradient.tolist()
        modify_file_with_gradient(nome_file, 'output.dmx', gradient)

        # risoluzione del problema lineare (ricerca di argmin) con MCF solver
        FILENAME = 'output3.dmx'
        f = open(FILENAME,'r')
        inputStr = f.read()
        f.close()
        mcf = MCFSimplex()
        mcf.LoadDMX(inputStr)
        mcf.SolveMCF()
        if mcf.MCFGetStatus() == 0:
            if k % 100 == 0: # stampa output ogni 100 iterazioni 
                soluzione_ottima = mcf.MCFGetFO()
                print("Iterazione: {:>4}  Step size: {:>20}  Optimal solution: {:.6f}".format(k, alpha, soluzione_ottima))
        else:
            print( "Problem unfeasible!")
        vettore_soluzione = {}   
        sol_x = [0] * numero_archi
        for key in vettore_soluzione:
            if key <= 1000:
                sol_x[key-1] = vettore_soluzione[key]

        # calcolo delle x_bar e determinazione della direzione di ricerca
        x_bar = sol_x
        d = [a - b for a, b in zip(x_bar, x_old)] # direzione

        # STEP 2 : determinazione alpha, step size
        alpha= 2 / (2+k)
    
        # STEP 3 : aggiornamento della posizione
        x_new = []
        for i in range(len(x_old)):
            x_new.append(x_old[i] + alpha * d[i])

        # check terminazione 
        gradient_per_check=(2 * np.dot(Q, x_new) ) + q
        prodotto_scalare = np.dot(gradient_per_check, d)
    
        # aggiornamento poszione e incremento numero di iterazioni
        x_old = x_new
        k += 1
        
    end_time = time.time()
    tempo_tot = end_time - start_time

    print()
    print('Prodotto scalare: {:>10.2f}  Step size finale: {:>10.8f}  Numero iterazioni totali: {:>4}'.format(prodotto_scalare, alpha, k))
    print('Tempo totale: {:>10.2f} secondi'.format(tempo_tot))
    return 

In [43]:
def pipeline(nome_file_dmx, nome_file_qfc):

    Q = matrice_Q(nome_file_qfc)[0] 
    numero_nodi, numero_archi, u, b, q, edges, from_ , to_ = leggi_file_dimacs(nome_file_dmx)
    E = np.zeros((numero_nodi, len(edges)), int)
    algoritmo(nome_file_dmx, epsilon = 0.01, max_iter = 500, Q = Q, q = q, u = u, numero_archi = numero_archi)
    return 

In [44]:
# prova di utilizzo 
pipeline(nome_file_dmx = '1000/netgen-1000-1-1-a-a-s.dmx', nome_file_qfc = "1000/netgen-1000-1-1-a-a-s.qfc")

Iterazione:    0  Step size:                  1.0  Optimal solution: 558624.000000
Iterazione:  100  Step size: 0.019801980198019802  Optimal solution: 558624.000000
Iterazione:  200  Step size: 0.009950248756218905  Optimal solution: 558624.000000
Iterazione:  300  Step size: 0.006644518272425249  Optimal solution: 558624.000000
Iterazione:  400  Step size: 0.004987531172069825  Optimal solution: 558624.000000

Prodotto scalare:       0.00  Step size finale: 0.00399202  Numero iterazioni totali:  500
Tempo totale:      10.55 secondi
